# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>

#### <center> **Final Project: Batch Processing** </center>
---

**Date**: October, 2025

**Student Name**:

**Professor**: Pablo Camarillo Ramirez

# Introduction

With this dataset we will try to check for the patterns in mental health in adults

# Dataset

I chose this dataset from kaggle from: https://www.kaggle.com/datasets/alamshihab075/mental-health-dataset with a column-family data model and a data schema with: magnitud,  significance, number of seismic monitoring stations, distance to nearest seismic station and earthquake focal depth

In [1]:
import findspark
findspark.init()
from pcamarillor.spark_utils import SparkUtils
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ML: Decision Trees & Random Forest") \
    .master("spark://spark-master:7077") \
    .config("spark.jars", "/opt/spark/work-dir/jars/postgresql-42.7.8.jar") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("INFO")

# Optimization (reduce the number of shuffle partitions)
spark.conf.set("spark.sql.shuffle.partitions", "5")



25/10/26 20:32:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
tsunami_schema_columns = [("index", "int"), 
     ("magnitud", "float"), 
     ("sig", "int"),
     ("nst", "int"),
     ("dmin", "float"),
     ("depth", "float"),
     ]
tsunami_schema = SparkUtils.generate_schema(tsunami_schema_columns)

tsunami_df = spark.read \
                .option("header", "true") \
                .schema(tsunami_schema) \
                .csv("/opt/spark/work-dir/data/tsunami/earthquake_data_tsunami.csv")

tsunami_df.printSchema()



root
 |-- index: integer (nullable = true)
 |-- magnitud: float (nullable = true)
 |-- sig: integer (nullable = true)
 |-- nst: integer (nullable = true)
 |-- dmin: float (nullable = true)
 |-- depth: float (nullable = true)



# Transformations and Actions

In [3]:
tsunami_df = tsunami_df.dropna()
tsunami_df = tsunami_df.drop("index")

# Persistence Data

In [5]:
jdbc_url = "jdbc:postgresql://postgres-iteso:5432/postgres"
table_name = "tsunami_transformed"

tsunami_df.write\
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", table_name) \
    .option("user", "postgres") \
    .option("password", "Admin@1234") \
    .option("driver", "org.postgresql.Driver") \
    .mode("overwrite") \
    .save()

print("DataFrame successfully written into a PosgreSQL DB !")

[Stage 0:>                                                          (0 + 1) / 1]

DataFrame successfully written into a PosgreSQL DB !


# DAG

In [6]:
jdbc_url = "jdbc:postgresql://postgres-iteso:5432/postgres"
db_properties = {
      "user": "postgres",      
      "password": "Admin@1234",
      "driver": "org.postgresql.Driver"
  }

df = spark.read \
    .jdbc(url=jdbc_url, table=table_name, properties=db_properties)

df.printSchema()
df.show(5, truncate=False)

root
 |-- magnitud: float (nullable = true)
 |-- sig: integer (nullable = true)
 |-- nst: integer (nullable = true)
 |-- dmin: float (nullable = true)
 |-- depth: float (nullable = true)

+--------+---+----+-----+-----+
|magnitud|sig|nst |dmin |depth|
+--------+---+----+-----+-----+
|8.0     |7  |768 |117.0|0.509|
|3.0     |3  |755 |147.0|3.125|
|4.0     |3  |755 |142.0|4.578|
|7.0     |5  |761 |192.0|2.977|
|9.0     |8  |1351|152.0|5.276|
+--------+---+----+-----+-----+
only showing top 5 rows


In [6]:
sc.stop()